In [1]:
# Cell 1: 导入必要的包
import torch
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np
from ppq.api import espdl_quantize_onnx, get_target_platform
from ppq.core import TargetPlatform, QuantizationStates, QuantizationPolicy, QuantizationProperty
from ppq.api.setting import QuantizationSettingFactory
from ppq.parser.espdl.espdl_typedef import ExporterPatternInfo
from ppq.IR import QuantableOperation


    ___________ ____        ____  ____  ____ 
   / ____/ ___// __ \      / __ \/ __ \/ __ \
  / __/  \__ \/ /_/ /_____/ /_/ / /_/ / / / /
 / /___ ___/ / ____/_____/ ____/ ____/ /_/ / 
/_____//____/_/         /_/   /_/    \___\_\ 




In [2]:
# Cell 2: 准备校准数据集
class CalibrationDataset(Dataset):
    def __init__(self, X):
        self.X = torch.FloatTensor(X).unsqueeze(1)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx]

# 加载校准数据
with open('../dataset/cal.pkl', 'rb') as f:
    X_cal, _ = pickle.load(f)

cal_dataset = CalibrationDataset(X_cal)
cal_loader = DataLoader(cal_dataset, batch_size=32, shuffle=False)

In [4]:
print(dir(graph))

NameError: name 'graph' is not defined

In [ ]:
import pprint

def pretty_print_graph(graph_dict):
    def print_dict(d, indent=0):
        for key, value in d.items():
            prefix = ' ' * indent
            if isinstance(value, dict):
                print(f"{prefix}{key}: {{")
                print_dict(value, indent + 4)
                print(f"{prefix}}}")
            else:
                if isinstance(value, list):
                    print(f"{prefix}{key}: [")
                    for item in value:
                        print(f"{' ' * (indent + 4)}{item},")
                    print(f"{prefix}]")
                else:
                    print(f"{prefix}{key}: {value}")

    print("Graph Attributes:\n")
    if isinstance(graph_dict, dict):
        print_dict(graph_dict)
    else:
        print("The provided input is not a dictionary.")

# Assuming graph.__dict__ has been assigned to the variable 'graph_dict'
graph_dict = graph.__dict__

# Use the function to print the graph attributes in a formatted way
pretty_print_graph(graph_dict)

In [7]:
# Cell 3: 配置混合精度量化参数并执行量化
ONNX_MODEL_PATH = "../training/gesture_model.onnx"
EXPORT_PATH = "../deployment/model/gesture_model_mixed.espdl"
input_shape = [1, 1, 96, 96]

# 设置混合精度量化配置
setting = QuantizationSettingFactory.espdl_setting()

# 对高误差层使用16位量化
high_error_layers = [
    "/first/first.0/Conv"
]

for layer in high_error_layers:
    setting.dispatching_table.append(layer, get_target_platform("esp32s3", 32))

setting.policy = QuantizationPolicy(
    QuantizationProperty.SYMMETRICAL + 
    QuantizationProperty.LINEAR +
    QuantizationProperty.PER_TENSOR +
    QuantizationProperty.POWER_OF_2
)

info = ExporterPatternInfo()
graph.set_extension_attrib('espdl_pattern_info', info)

for op in graph.operations.values():
    if isinstance(op, QuantableOperation):
        for config, var in op.config_with_variable:
            if config.state != QuantizationStates.FP32:
                if config.scale is not None:
                    # 计算每个scale对应的exponent
                    # print(config.scale)
                    exponent = [-int(torch.floor(torch.log2(config.scale)))]
                    info.add_var_exponents(var.name, exponent)
                else:
                    info.add_var_exponents(var.name, [0])

# 为所有变量设置默认exponent
for var in graph.variables.values():
    if not info.get_var_exponents(var.name):
        info.add_var_exponents(var.name, [0])

# 执行量化
graph = espdl_quantize_onnx(
    onnx_import_file=ONNX_MODEL_PATH,
    espdl_export_file=EXPORT_PATH,
    calib_dataloader=cal_loader,
    calib_steps=50,
    input_shape=[input_shape],
    target="esp32s3",
    num_of_bits=8,
    setting=setting,
    device="cpu",
    error_report=True,
    verbose=1
)

[WARNING][ESPDL][2024-12-02 20:45:12]:  Do not support num_of_bits:32, will change to TargetPlatform.FP32
[20:45:12] PPQ Quantization Fusion Pass Running ...       Finished.
[20:45:12] PPQ Quantize Simplify Pass Running ...         Finished.
[20:45:12] PPQ Parameter Quantization Pass Running ...    Finished.
[20:45:12] PPQ Runtime Calibration Pass Running ...       

Calibration Progress(Phase 2): 100%|████████████| 50/50 [00:04<00:00, 10.24it/s]


Finished.
[20:45:21] PPQ Quantization Alignment Pass Running ...    Finished.
[20:45:21] PPQ Passive Parameter Quantization Running ... Finished.
--------- Network Snapshot ---------
Num of Op:                    [27]
Num of Quantized Op:          [26]
Num of Variable:              [74]
Num of Quantized Var:         [71]
------- Quantization Snapshot ------
Num of Quant Config:          [98]
ACTIVATED:                    [29]
OVERLAPPED:                   [32]
PASSIVE:                      [37]
Network Quantization Finished.


Analysing Graphwise Quantization Error(Phrase 1):: 100%|█| 8/8 [00:00<00:00,  8.
Analysing Graphwise Quantization Error(Phrase 2):: 100%|█| 8/8 [00:01<00:00,  4.


Layer                              | NOISE:SIGNAL POWER RATIO 
/classifier/classifier.2/Gemm:     | ████████████████████ | 15.570%
/layers/layers.3/conv/conv.6/Conv: | ████████████████     | 12.826%
/layers/layers.3/conv/conv.3/Conv: | ████████             | 7.020%
/layers/layers.2/conv/conv.6/Conv: | ███████              | 6.491%
/layers/layers.3/conv/conv.0/Conv: | ██████               | 6.142%
/layers/layers.2/conv/conv.3/Conv: | ███                  | 4.065%
/layers/layers.1/conv/conv.6/Conv: | ███                  | 3.711%
/layers/layers.0/conv/conv.3/Conv: | ██                   | 3.312%
/layers/layers.1/conv/conv.3/Conv: | ██                   | 3.204%
/layers/layers.2/conv/conv.0/Conv: | ██                   | 3.012%
/layers/layers.0/conv/conv.6/Conv: | █                    | 2.253%
/layers/layers.1/conv/conv.0/Conv: | █                    | 2.236%
/layers/layers.0/conv/conv.0/Conv: |                      | 1.632%


Analysing Layerwise quantization error:: 100%|██| 13/13 [00:13<00:00,  1.04s/it]

Layer                              | NOISE:SIGNAL POWER RATIO 
/layers/layers.0/conv/conv.0/Conv: | ████████████████████ | 1.031%
/layers/layers.0/conv/conv.3/Conv: | █████████████        | 0.698%
/layers/layers.2/conv/conv.3/Conv: | ████████████         | 0.625%
/layers/layers.3/conv/conv.3/Conv: | ████                 | 0.214%
/layers/layers.0/conv/conv.6/Conv: | ███                  | 0.140%
/layers/layers.1/conv/conv.0/Conv: | ██                   | 0.129%
/layers/layers.1/conv/conv.3/Conv: | █                    | 0.084%
/layers/layers.1/conv/conv.6/Conv: | █                    | 0.068%
/layers/layers.2/conv/conv.6/Conv: | █                    | 0.063%
/layers/layers.3/conv/conv.0/Conv: | █                    | 0.046%
/classifier/classifier.2/Gemm:     | █                    | 0.042%
/layers/layers.2/conv/conv.0/Conv: | █                    | 0.040%
/layers/layers.3/conv/conv.6/Conv: |                      | 0.012%
[INFO][ESPDL][2024-12-02 20:45:37]:  skip not QuantableOperation
[

In [ ]:
# Cell 4: 评估量化后的模型
from ppq.executor import TorchExecutor
import time
from tqdm import tqdm

def evaluate_quantized_model(graph, test_loader, y_test):
    """
    评估量化后的模型性能
    """
    executor = TorchExecutor(graph=graph, device='cpu')
    total_time = 0
    correct = 0
    total = 0
    
    print("\n开始评估量化模型性能...")
    with torch.no_grad():
        for batch in tqdm(test_loader):
            start = time.time()
            outputs = executor.forward(inputs=batch)
            total_time += (time.time() - start)
            
            # 计算准确率
            _, predicted = torch.max(outputs[0], 1)
            total += batch.size(0)
            correct += (predicted == y_test[total-batch.size(0):total]).sum().item()

    # 计算并打印结果
    avg_time = (total_time / len(test_loader)) * 1000  # 转换为毫秒
    accuracy = (correct / total) * 100

    print(f"\n评估结果:")
    print(f"平均推理时间: {avg_time:.2f} ms")
    print(f"模型准确率: {accuracy:.2f}%")
    
    return avg_time, accuracy

# 加载测试数据
with open('../dataset/test.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

# 准备测试数据集
test_dataset = CalibrationDataset(X_test)  # 复用之前的Dataset类
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 执行评估
evaluate_quantized_model(graph, test_loader, y_test)

In [6]:
# 遍历图中的所有操作
for op in graph.operations.values():
    print(f"\nOperation: {op.name}")
    # 打印输入形状
    for inp in op.inputs:
        print(f"Input '{inp.name}' shape: {inp.shape}")
    # 打印输出形状  
    for out in op.outputs:
        print(f"Output '{out.name}' shape: {out.shape}")


Operation: /first/first.0/Conv
Input 'input' shape: [1, 1, 96, 96]
Input 'onnx::Conv_140' shape: [16, 1, 3, 3]
Input 'onnx::Conv_141' shape: [16]
Output '/first/first.0/Conv_output_0' shape: [1, 16, 48, 48]

Operation: /first/first.2/Clip
Input '/first/first.0/Conv_output_0' shape: [1, 16, 48, 48]
Input '/first/first.2/Constant_output_0' shape: []
Input '/first/first.2/Constant_1_output_0' shape: []
Output '/first/first.2/Clip_output_0' shape: [1, 16, 48, 48]

Operation: /layers/layers.0/conv/conv.0/Conv
Input '/first/first.2/Clip_output_0' shape: [1, 16, 48, 48]
Input 'onnx::Conv_143' shape: [96, 16, 1, 1]
Input 'onnx::Conv_144' shape: [96]
Output '/layers/layers.0/conv/conv.0/Conv_output_0' shape: [1, 96, 48, 48]

Operation: /layers/layers.0/conv/conv.2/Clip
Input '/layers/layers.0/conv/conv.0/Conv_output_0' shape: [1, 96, 48, 48]
Input '/layers/layers.0/conv/conv.2/Constant_output_0' shape: []
Input '/layers/layers.0/conv/conv.2/Constant_1_output_0' shape: []
Output '/layers/layers

In [10]:
# 查看第一层量化配置
first_conv = graph.operations['/first/first.0/Conv']
if isinstance(first_conv, QuantableOperation):
   for config, var in first_conv.config_with_variable:
       print(f"\nVariable: {var.name}")
       print(f"State: {config.state}")
       print(f"Platform: {config.platform}")
       print(f"Bit width: {config.num_of_bits}")
       print(f"Scale: {config.scale}")
       print(f"Offset: {config.offset}")

In [ ]:
!ls *.png

In [ ]:
# Cell 5: 使用量化后的模型识别一张图片
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_image(image_path):
    """
    图片预处理函数
    1. 读取图片
    2. 转换为灰度图
    3. 调整大小
    4. 归一化
    """
    # 读取图片
    img = cv2.imread(image_path)
    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 调整大小
    TARGET_SIZE = (96, 96)
    resized = cv2.resize(gray, TARGET_SIZE)
    # 归一化
    normalized = resized.astype('float32') / 255.0
    # 添加 batch 和 channel 维度
    image_tensor = torch.FloatTensor(normalized).unsqueeze(0).unsqueeze(0)
    
    return image_tensor, img

def predict_image(graph, image_tensor):
    """
    使用量化后的模型预测图像的类别置信度。
    """
    executor = TorchExecutor(graph=graph, device='cpu')
    with torch.no_grad():
        outputs = executor.forward(inputs=[image_tensor])
        confidences = torch.softmax(outputs[0], dim=1).numpy().flatten()
    
    return confidences

# 定义手势类别
gesture_labels = {
    '00': 'palm',
    '01': 'l',
    '02': 'fist',
    '03': 'thumb',
    '04': 'index',
    '05': 'ok',
    '06': 'c',
    '07': 'down'
}

# 加载并预处理图像
# 请将 "your_image.jpg" 替换为你要预测的图像的实际路径，例如："/path/to/your_image.jpg"
image_path = "frame_00_01_0009.png"  # 这里将 "your_image.jpg" 替换为你要使用的图像路径
image_tensor, original_image = preprocess_image(image_path)

# 使用量化后的模型进行预测
confidences = predict_image(graph, image_tensor)

# 打印模型输出的各个置信度，并关联类别标签
print("\n预测结果: \n")
for idx, confidence in enumerate(confidences):
    label = gesture_labels[f'{idx:02}']
    print(f"类别: {label}, 置信度: {confidence:.4f}")

# 显示原始图片
plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')
plt.show()